In [1]:
# ================================
# SYSTEM SETUP AND DEPENDENCIES
# ================================

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✗ Not running in Google Colab")

# Mount Google Drive for persistent storage
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

# ================================
# MICROMAMBA ENVIRONMENT SETUP
# ================================
# Install micromamba in Google Colab
!wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# Make it easier to call
!mkdir -p /usr/local/bin
!mv bin/micromamba /usr/local/bin/micromamba

# Initialize micromamba shell (bash)
#!micromamba shell init -s bash  ~/micromamba

✓ Running in Google Colab
Mounted at /content/drive
bin/micromamba


In [2]:
# ================================
# LIGANDMPNN SETUP
# ================================

!git clone https://github.com/dauparas/LigandMPNN.git
%cd LigandMPNN
!bash get_model_params.sh "./model_params"

!micromamba create -y -n ligandmpnn -c conda-forge -c bioconda tmalign  python=3.11 git

!micromamba run -n ligandmpnn pip install -r requirements.txt
%cd /content

print("✓ LigandMPNN setup complete")

Cloning into 'LigandMPNN'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 338 (delta 120), reused 99 (delta 99), pack-reused 176 (from 1)
Receiving objects: 100% (338/338), 2.64 MiB | 3.58 MiB/s, done.
Resolving deltas: 100% (133/133), done.
/content/LigandMPNN
[+] 0.0s
[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  
bioconda/linux-64      1%
bioconda/noarch       ⣾  [+] 0.2s
conda-forge/linux-64   6%
conda-forge/noarch    13%
bioconda/linux-64     71%
bioconda/noarch       21%bioconda/linux-64                                 
[+] 0.3s
conda-forge/linux-64  11%
conda-forge/noarch    23%
bioconda/noarch       69%bioconda/noarch                                   
[+] 0.4s
conda-forge/linux-64  16%
conda-forge/noarch    33%[+] 0.5s
conda-forge/linux-64  25%
conda-forge/noarch    52%[+] 0.6s
conda-forge/linux-64  35%
conda-forge/noarch    74%[+] 0.7s
conda-forge/l

In [3]:
%cd /content

/content


In [4]:
# ================================
# BOLTZ SETUP
# ================================



%%bash
# --- Create env ---
micromamba create -y -n boltz python=3.10 -c conda-forge

# --- Install CUDA-enabled PyTorch ---
micromamba install -y -n boltz pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia -c conda-forge

# --- Activate environment and install Boltz + deps ---
micromamba run -n boltz bash -c set -e
runtime="GPU(L4 or T4)"
deps="numpy hydra-core pytorch-lightning rdkit dm-tree requests pandas types-requests einops einx fairscale mashumaro modelcif wandb click pyyaml biopython scipy numba gemmi scikit-learn chembl_structure_pipeline cuequivariance_ops_cu12 cuequivariance_ops_torch_cu12 cuequivariance_torch"

if [[ "$runtime" == "GPU(L4 or T4)" ]]; then
  precision="32-true"
else
  precision="bf16-true"
fi

git clone https://github.com/jwohlwend/boltz.git /content/boltz
sed -i "s/bf16-mixed/${precision}/g" /content/boltz/src/boltz/main.py
pip install ipywidgets $deps
#cd /content/boltz && pip install --no-deps -e .
cd /content/boltz && pip install --no-deps .



echo "✅ Boltz environment setup complete."

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache


Transaction

  Prefix: /root/.local/share/mamba/envs/boltz

  Updating specs:

   - python=3.10


  Package               Version  Build               Channel           Size
─────────────────────────────────────────────────────────────────────────────
  Install:
─────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  conda_forge         conda-forge     Cached
  + _openmp_mutex           4.5  2_gnu               conda-forge     Cached
  + bzip2                 1.0.8  hda65f42_8          conda-forge     Cached
  + ca-certificates   2025.10.5  hbd8a1cb_0          conda-forge     Cached
  + icu                    75.1  he02047a_0          conda-forge     Cached
  + ld_impl_linux-64       2.44  h1aa0949_4          conda-forge     Cached
  + libexpat              2.7.1  hecca717_0          con

warning  libmamba [libjpeg-turbo-2.0.0-h9bf148f_0] The following files were already present in the environment:
    - bin/cjpeg
    - bin/djpeg
    - bin/jpegtran
    - bin/rdjpgcom
    - bin/wrjpgcom
    - include/jconfig.h
    - include/jerror.h
    - include/jmorecfg.h
    - include/jpeglib.h
    - lib/libjpeg.a
    - lib/libjpeg.so
    - lib/pkgconfig/libjpeg.pc
    - share/man/man1/cjpeg.1
    - share/man/man1/djpeg.1
    - share/man/man1/jpegtran.1
    - share/man/man1/rdjpgcom.1
    - share/man/man1/wrjpgcom.1
Cloning into '/content/boltz'...


In [5]:
import glob
import os
import sys

In [6]:
working_dir = '/content/drive/MyDrive/enzdes_rfd2_tutorial'
mpnn_outdir = f'{working_dir}/outputs'
pdbs_for_mpnn = f'{working_dir}/examples_for_ligandMPNN'
pdbs_for_mpnn = '/content/drive/MyDrive/clean_enzdes_rfd2_tutorial/examples_for_ligandMPNN'
pdbs = glob.glob(f'{pdbs_for_mpnn}/*pdb')
pdb = pdbs[0]
omit = 'CH'
cmd = f'python /content/LigandMPNN/run.py \
    --model_type ligand_mpnn --pdb_path {pdb} --out_folder {mpnn_outdir} \
    --pack_side_chains 1 --number_of_batches 3 --temperature 0.3 --omit_AA {omit} \
    --repack_everything 0   --checkpoint_path_sc /content/LigandMPNN/model_params/ligandmpnn_sc_v_32_002_16.pt \
    --checkpoint_ligand_mpnn /content/LigandMPNN/model_params/ligandmpnn_v_32_010_25.pt \n'
print(cmd)

python /content/LigandMPNN/run.py     --model_type ligand_mpnn --pdb_path /content/drive/MyDrive/clean_enzdes_rfd2_tutorial/examples_for_ligandMPNN/1qji_theozyme_HEHHMY_ORI_126_LENGTH_180_200_tutorial_t1_1_9-atomized-bb-False.pdb --out_folder /content/drive/MyDrive/enzdes_rfd2_tutorial/outputs     --pack_side_chains 1 --number_of_batches 3 --temperature 0.3 --omit_AA CH     --repack_everything 0   --checkpoint_path_sc /content/LigandMPNN/model_params/ligandmpnn_sc_v_32_002_16.pt     --checkpoint_ligand_mpnn /content/LigandMPNN/model_params/ligandmpnn_v_32_010_25.pt 



In [ ]:
%cd /content/LigandMPNN

/content/LigandMPNN


In [10]:
!micromamba run -n ligandmpnn python /content/LigandMPNN/run.py     --model_type ligand_mpnn --pdb_path /content/drive/MyDrive/clean_enzdes_rfd2_tutorial/examples_for_ligandMPNN/1qji_theozyme_HEHHMY_ORI_126_LENGTH_180_200_tutorial_t1_1_9-atomized-bb-False.pdb --out_folder /content/drive/MyDrive/enzdes_rfd2_tutorial/outputs     --pack_side_chains 1 --number_of_batches 3 --temperature 0.3 --omit_AA CH     --repack_everything 0   --checkpoint_path_sc /content/LigandMPNN/model_params/ligandmpnn_sc_v_32_002_16.pt     --checkpoint_ligand_mpnn /content/LigandMPNN/model_params/ligandmpnn_v_32_010_25.pt



/root/.local/share/mamba/envs/ligandmpnn/lib/python3.11/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
@> ProDy is configured: verbosity='none'
Designing protein from this path: /content/drive/MyDrive/clean_enzdes_rfd2_tutorial/examples_for_ligandMPNN/1qji_theozyme_HEHHMY_ORI_126_LENGTH_180_200_tutorial_t1_1_9-atomized-bb-False.pdb
These residues will be redesigned:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', '

In [11]:
import boltz